In [122]:
# Import relevant libraries to make HTTP requests and parse JSON response
import requests
import json
import csv
import io
import chembl_webresource_client as cwc
import pandas as pd

print("done")

done


In [124]:
# Set disease_id variable for desired disease
disease_id = "EFO_0005537"

# Build query string to get target information as well as count
query_string = """
query AssociatedTargets {
  disease(efoId: "EFO_0005537") {
    id
    name
    associatedTargets(page: { size: 3, index: 0 }) {
      rows {
        target {
          id
          approvedName
          dbXrefs{
              id
            }
        }
        score
      }
    }
  }
}
"""

# Set variables object of arguments to be passed to endpoint
variables = {"efoId": disease_id}

# Set base URL of GraphQL API endpoint
base_url = "https://api.platform.opentargets.org/api/v4/graphql"

# Perform POST request and check status code of response
r = requests.post(base_url, json={"query": query_string, "variables": variables})
print(r.status_code)

#Transform API response from JSON into Python dictionary and print in console
api_response = json.loads(r.text)
print('done')
print(api_response)

200
done
{'data': {'disease': {'id': 'EFO_0005537', 'name': 'triple-negative breast cancer', 'associatedTargets': {'rows': [{'target': {'id': 'ENSG00000184292', 'approvedName': 'tumor associated calcium signal transducer 2', 'dbXrefs': [{'id': '11530'}, {'id': '2MAE'}, {'id': '2MVK'}, {'id': '2MVL'}, {'id': '7E5M'}, {'id': '7E5N'}, {'id': '7PEE'}, {'id': 'CHEMBL3856163'}, {'id': 'DB12893'}, {'id': 'IPR043406'}, {'id': 'IPR000716'}, {'id': 'IPR036857'}, {'id': 'SignalP-noTM'}]}, 'score': 0.43149003913252654}, {'target': {'id': 'ENSG00000198900', 'approvedName': 'DNA topoisomerase I', 'dbXrefs': [{'id': '11986'}, {'id': '1A31'}, {'id': '1A35'}, {'id': '1A36'}, {'id': '1EJ9'}, {'id': '1K4S'}, {'id': '1K4T'}, {'id': '1LPQ'}, {'id': '1NH3'}, {'id': '1R49'}, {'id': '1RR8'}, {'id': '1RRJ'}, {'id': '1SC7'}, {'id': '1SEU'}, {'id': '1T8I'}, {'id': '1TL8'}, {'id': 'CHEMBL1781'}, {'id': 'DB07354'}, {'id': 'DB08159'}, {'id': 'DB05482'}, {'id': 'DB04690'}, {'id': 'DB05806'}, {'id': 'DB00970'}, {'id'

In [125]:
a=0
#create list of target IDs with associated evidence scores by calling them from dictionary
target_info = api_response['data']['disease']['associatedTargets']['rows']
#print(target_info)


target_id_list = []
#pull IDs and scores from dictionary and add to new list
while a < len(target_info):
    print(a)

    for dict in target_info[a]['target']['dbXrefs']:
        #print(dict)
        # Get rid of all IDs that aren't CHEMBL IDs, then put CHEMBL ID in list to add to tuple
        newDict = {key:val for key, val in (dict).items() if "CHEMBL" in val}
        if len(newDict) == 0:
            continue
        chembl_id = newDict['id']
    target_id_list.append((target_info[a]['target'].get('id'), chembl_id, target_info[a]['score']))
    a+=1
print(target_id_list)

0
1
2
[('ENSG00000184292', 'CHEMBL3856163', 0.43149003913252654), ('ENSG00000198900', 'CHEMBL1781', 0.42420687638099397), ('ENSG00000120217', 'CHEMBL3580522', 0.40556949639812895)]


In [137]:
from chembl_webresource_client.new_client import new_client

def getTargetInfo(targets, info)

target = new_client.target
activity = new_client.activity

# Create targets_dict, which will be a dictionary containing target names, with compounds and compound data associated with that target.
targets_dict = {}

# Create darget_data, which will be a dictionary containing target names, with dataframes with compound data associated with that target
target_data_list = []

#For each target pulled from OpenTargets:
for target in target_id_list:
    target_data = {}
    print(target[0])
    # Get compound activitiy data from Chembl for each target, including only exact IC50 values. Sort by IC50 value.
    activities = activity.filter(target_chembl_id=target[1], standard_type="IC50", relation = '=').order_by('standard_value')
    #print(activities)
    
    compound_dictionary = {}
    
    #create seperate list for each type of data
    smiles_list = []
    standard_value_list = []
    compound_list = []
    type_list = []
    assays_list = []

    
    
    for compound in activities:
        #print(compound)
        slimDict = {key:val for key, val in compound.items() if key == 'type' or key == 'canonical_smiles' or key == 'standard_value'}
        

        #slimDict = {key:val for key, val in compound.items() if key == 'value'}

        
        compound_id = compound['molecule_chembl_id']
        compound_list.append(compound_id)
        type_list.append(compound['type'])
        standard_value_list.append(compound['standard_value'])
        smiles_list.append(compound['canonical_smiles'])
        assays_list.append(compound['assay_chembl_id'])        
        #print(compound_id)
    
        compound_dictionary[compound['molecule_chembl_id']] = slimDict
        df = pd.DataFrame.from_dict(compound_dictionary)
    
    
    targets_dict[target[0]] = compound_dictionary

    
    to_pandas_dict = {"compound id": compound_list, "smiles": smiles_list, "type" : type_list, "standard value": standard_value_list, 'assay_id': assays_list}
    pandas_df = pd.DataFrame.from_dict(to_pandas_dict)
    target_data['target_id'] = target[0]
    target_data['data'] = pandas_df
    target_data_list.append(target_data)
    





ENSG00000184292
ENSG00000198900
ENSG00000120217


In [151]:

for target in target_data_list:
    print(target['target_id'])

    display(target['data'])
    #display(target['data'])


ENSG00000184292


,compound id,smiles,type,standard value,assay_id


ENSG00000198900


,compound id,smiles,type,standard value,assay_id
0,CHEMBL3968023,C[n+]1ccc2c3c(nc4cc5c(cc42)OCO5)-c2ccccc2-c31....,IC50,0.45,CHEMBL3867357
1,CHEMBL3914791,C[n+]1c2c3c(nc(-c4ccccc4)cc3c3ccccc31)-c1ccccc...,IC50,0.475,CHEMBL3867357
2,CHEMBL3896867,[Br-].c1ccc(C[n+]2ccc3c4c(nc5cc6c(cc53)OCO6)-c...,IC50,0.52,CHEMBL3867357
3,CHEMBL3889816,[Br-].c1ccc(C[n+]2ccc3c4c(nc5ccccc53)-c3ccccc3...,IC50,0.57,CHEMBL3867357
4,CHEMBL3984603,Cc1cc2c3c(nc4ccccc42)-c2ccccc2-c3[n+]1C.[I-],IC50,0.7,CHEMBL3867357
...,...,...,...,...,...
451,CHEMBL426092,O=C1Oc2ccc(O)cc2/C1=C\c1ccc(O)cc1,IC50,555000.0,CHEMBL827155
452,CHEMBL9164,CCCCCCC(=O)O[C@H](CC=C(C)C)C1=CC(=O)c2c(O)ccc(...,IC50,625000.0,CHEMBL663903
453,CHEMBL372372,COc1cc(/C=C2/C(=O)Oc3ccccc32)ccc1O,IC50,630000.0,CHEMBL827155
454,CHEMBL381195,COc1ccc(/C=C2/C(=O)Oc3ccccc32)cc1O,IC50,800000.0,CHEMBL827155


ENSG00000120217


,compound id,smiles,type,standard value,assay_id
0,CHEMBL4776444,Cc1c(COc2cc(OCc3cncc(C#N)c3)c(CNC(C)(C)C(=O)O)...,IC50,1.0,CHEMBL4775268
1,CHEMBL4081869,COc1cc(OCc2cccc(-c3ccccc3)c2C)cc(OC)c1CNCCNC(C)=O,IC50,1.0,CHEMBL4775269
2,CHEMBL4776700,Cn1cnc(S(=O)(=O)N2CCCC3=C[C@H]4C[C@H](CN5CCCC[...,IC50,1.0,CHEMBL4775268
3,CHEMBL4785255,Cc1c(COc2cc(OCc3cncc(C#N)c3)c(CN[C@](C)(CO)C(=...,IC50,1.0,CHEMBL4775268
4,CHEMBL5171101,CCCC[C@H]1C(=O)N(C)[C@@H](CCCC)C(=O)N[C@@H](C)...,IC50,1.03,CHEMBL5096762
5,CHEMBL4071326,Cc1c(COc2cc(OCc3cccc(C#N)c3)c(CN3C[C@H](O)C[C@...,IC50,1.4,CHEMBL4017391
6,CHEMBL4084368,Cc1cc(CN[C@H](CO)C(=O)O)c(OCc2cccc(C#N)c2)cc1O...,IC50,2.25,CHEMBL4017391
7,CHEMBL4288470,CCCC[C@H]1C(=O)N(C)[C@@H](CCCC)C(=O)N[C@@H](CC...,IC50,5.2,CHEMBL4263391
8,CHEMBL3582254,Cc1c(COc2ccc(CN[C@H](C)C(=O)O)cc2Cl)cccc1-c1cc...,IC50,6.0,CHEMBL3583018
9,CHEMBL4061613,COc1cc(OCc2cccc(-c3ccccc3)c2C)cc(OC)c1CN[C@@H]...,IC50,6.0,CHEMBL4017391


In [154]:
smiles = target_data_list[1]['data']['smiles']
print(smiles)
smiles_list = []
for row in smiles:
    smiles_list.append(row)
print(smiles_list)

0      C[n+]1ccc2c3c(nc4cc5c(cc42)OCO5)-c2ccccc2-c31....
1      C[n+]1c2c3c(nc(-c4ccccc4)cc3c3ccccc31)-c1ccccc...
2      [Br-].c1ccc(C[n+]2ccc3c4c(nc5cc6c(cc53)OCO6)-c...
3      [Br-].c1ccc(C[n+]2ccc3c4c(nc5ccccc53)-c3ccccc3...
4           Cc1cc2c3c(nc4ccccc42)-c2ccccc2-c3[n+]1C.[I-]
                             ...                        
451                    O=C1Oc2ccc(O)cc2/C1=C\c1ccc(O)cc1
452    CCCCCCC(=O)O[C@H](CC=C(C)C)C1=CC(=O)c2c(O)ccc(...
453                   COc1cc(/C=C2/C(=O)Oc3ccccc32)ccc1O
454                   COc1ccc(/C=C2/C(=O)Oc3ccccc32)cc1O
455                       O=C1Oc2ccc(O)cc2/C1=C\c1ccccc1
Name: smiles, Length: 456, dtype: object
['C[n+]1ccc2c3c(nc4cc5c(cc42)OCO5)-c2ccccc2-c31.[I-]', 'C[n+]1c2c3c(nc(-c4ccccc4)cc3c3ccccc31)-c1ccccc1-2.[I-]', '[Br-].c1ccc(C[n+]2ccc3c4c(nc5cc6c(cc53)OCO6)-c3ccccc3-c42)cc1', '[Br-].c1ccc(C[n+]2ccc3c4c(nc5ccccc53)-c3ccccc3-c42)cc1', 'Cc1cc2c3c(nc4ccccc42)-c2ccccc2-c3[n+]1C.[I-]', 'CC[n+]1ccc2c3c(nc4ccccc42)-c2ccccc2-c31.[I-]', 

In [164]:
smiles_list = str(smiles_list)



#with open(target_data_list[1]['target_id']+'Smiles.txt','w') as data: 
      #data.write(str(smiles_list).replace(',', '\n').replace("'", ""))

TypeError: string indices must be integers